In [1]:
from nltk.corpus import movie_reviews

In [2]:
movie_reviews.categories()

['neg', 'pos']

In [3]:
# movie_reviews.fileids('neg')

In [4]:
movie_reviews.words(movie_reviews.fileids('neg')[5])

['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...]

In [5]:
documents=[]
for category in movie_reviews.categories():
    for file in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(file),category))

In [6]:
documents[0:5]

[(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg'),
 (['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg'),
 (['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...], 'neg'),
 (['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...], 'neg'),
 (['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...], 'neg')]

In [7]:
import random
random.shuffle(documents)

In [8]:
documents[0:5]

[(['if', 'i', 'were', 'one', 'of', 'those', 'arrogant', ...], 'neg'),
 (['in', 'french', ',', 'the', 'phrase', '"', 'film', ...], 'neg'),
 (['what', 'are', 'the', 'warning', 'signs', 'of', 'a', ...], 'neg'),
 (['"', 'be', 'gentle', ',', '"', 'urges', 'natasha', ...], 'neg'),
 (['every', 'once', 'in', 'a', 'while', ',', 'when', ...], 'pos')]

In [9]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.corpus import wordnet
import nltk

In [10]:
stops=set(stopwords.words('english'))
punct=list(string.punctuation)
stops.update(punct)
len(stops)

211

In [11]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [12]:
lemmatizer=WordNetLemmatizer()

In [13]:
w='better'
print(pos_tag(w))
print(pos_tag([w]))
lemmatizer.lemmatize(w,get_simple_pos(pos_tag([w])[0][1]))

[('b', 'NN'), ('e', 'NN'), ('t', 'NN'), ('t', 'NN'), ('e', 'NN'), ('r', 'NN')]
[('better', 'RBR')]


'well'

In [14]:
def clean_review(words):
    output_words=[]
    for word in words:
        if word.lower() not in stops:
            pos=pos_tag([word])
            clean_word=lemmatizer.lemmatize(word,get_simple_pos(pos[0][1]))
            output_words.append(clean_word)
    return output_words

In [15]:
documents=[(clean_review(document),category) for document,category in documents]

In [16]:
training_doc=documents[:1500]
testing_doc=documents[1500:]

In [17]:
all_words=[]
for document in training_doc:
    all_words+=document[0]

In [18]:
dictionary= nltk.FreqDist(all_words)

In [19]:
common=dictionary.most_common(3000)
features=[i[0] for i in common]

In [1]:
def get_feature_dict(words):
    current_features={}
    word_set=set(words)
    for w in features:
        current_features[w]=w in word_set
    return current_features

In [21]:
training_data=[(get_feature_dict(doc),category) for doc,category in training_doc]

In [22]:
testing_data=[(get_feature_dict(doc),category) for doc,category in testing_doc]

In [23]:
from nltk import NaiveBayesClassifier

In [24]:
clf=NaiveBayesClassifier.train(training_data)

In [25]:
nltk.classify.accuracy(clf,testing_data)

0.78

In [26]:
clf.show_most_informative_features(15)

Most Informative Features
               stupidity = True              neg : pos    =     15.9 : 1.0
                  seagal = True              neg : pos    =     12.0 : 1.0
              schumacher = True              neg : pos    =      9.4 : 1.0
                    lame = True              neg : pos    =      8.9 : 1.0
                  alicia = True              neg : pos    =      8.8 : 1.0
             outstanding = True              pos : neg    =      8.3 : 1.0
                   jolie = True              neg : pos    =      8.1 : 1.0
               ludicrous = True              neg : pos    =      7.7 : 1.0
               painfully = True              neg : pos    =      7.4 : 1.0
             wonderfully = True              pos : neg    =      7.0 : 1.0
              whatsoever = True              neg : pos    =      6.8 : 1.0
                   damon = True              pos : neg    =      6.6 : 1.0
                   crowe = True              pos : neg    =      6.5 : 1.0

In [30]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import RandomForestClassifier

In [32]:
svc=SVC()
classifier_sklearn=SklearnClassifier(svc)

In [33]:
clf_svc=classifier_sklearn.train(training_data)

In [34]:
nltk.classify.accuracy(clf_svc,testing_data)

0.748

In [36]:
rfc=RandomForestClassifier()
classifier_sklearn1=SklearnClassifier(rfc)

In [39]:
clf_rfc=classifier_sklearn1.train(training_data)

In [42]:
nltk.classify.accuracy(clf_rfc,testing_data)

0.662

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
training_set={"the sky is blue","the weather is awesome"}
cv=CountVectorizer(max_features=3)
a=cv.fit_transform(training_set)
a.todense()


matrix([[1, 1, 1],
        [0, 1, 1]], dtype=int64)

In [52]:
" ".join(training_set)

'the weather is awesome the sky is blue'

In [57]:
cv.get_feature_names()

['awesome', 'is', 'the']

In [58]:
categories=[category for document,category in documents]

In [59]:
text_documents=[" ".join(document) for document,category in documents]

In [61]:
from sklearn.model_selection  import train_test_split

In [62]:
x_train,x_test,y_train,y_test=train_test_split(text_documents,categories)

In [83]:
cv=CountVectorizer(max_features=2000,ngram_range=(1,2))

In [84]:
x_train_features=cv.fit_transform(x_train)

In [85]:
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 2, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [86]:
cv.get_feature_names()

['000',
 '10',
 '100',
 '13',
 '15',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '20',
 '30',
 '50',
 '60',
 '70',
 '80',
 '90',
 'abandon',
 'ability',
 'able',
 'absolutely',
 'academy',
 'accent',
 'accept',
 'accident',
 'accomplish',
 'achieve',
 'achievement',
 'across',
 'act',
 'action',
 'action film',
 'action movie',
 'action scene',
 'action sequence',
 'actor',
 'actress',
 'actual',
 'actually',
 'ad',
 'adam',
 'adaptation',
 'add',
 'addition',
 'admit',
 'adult',
 'adventure',
 'affair',
 'affleck',
 'african',
 'age',
 'agent',
 'ago',
 'agree',
 'agrees',
 'ahead',
 'aid',
 'aim',
 'air',
 'al',
 'ala',
 'alan',
 'alex',
 'alien',
 'alive',
 'allen',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'along way',
 'already',
 'also',
 'although',
 'always',
 'amaze',
 'america',
 'american',
 'among',
 'amount',
 'amuse',
 'amy',
 'anderson',
 'andrew',
 'angel',
 'angle',
 'angry',
 'animal',
 'animate',
 'animation',
 'anna',
 'anne',
 'annie',
 'annoy',
 'anot

In [87]:
x_test_features=cv.transform(x_test)

In [88]:
x_test_features.shape

(500, 2000)

In [89]:
svc=SVC()

In [90]:
svc.fit(x_train_features,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [91]:
svc.score(x_test_features,y_test)

0.818